In [3]:
import pandas as pd
import numpy as np
np.set_printoptions(suppress=True, precision=3, threshold=np.inf)

road = pd.read_csv('data\\road_num.csv', delimiter=',')
x_road = road.drop(columns=['Price', 'ID'])
y_road = road['Price']

In [4]:
from sklearn.model_selection import train_test_split

road_train, road_test, y_train, y_test = train_test_split(x_road, y_road, test_size=0.2, random_state=42)

y_train.to_csv('temp1.csv')
road_train = road_train.to_numpy()
road_test = road_test.to_numpy()
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

y_train = y_train.reshape(-1,1)
y_test = y_test.reshape(-1,1)

road_train = road_train[:,1:]
road_test = road_test[:,1:]

road_train.shape, road_test.shape, y_train.shape, y_test.shape

((1860, 6), (465, 6), (1860, 1), (465, 1))

In [5]:
road_train

array([[0.   , 0.9  , 0.9  , 0.   , 0.5  , 0.311],
       [1.   , 0.9  , 0.7  , 0.   , 1.   , 1.   ],
       [1.   , 0.9  , 1.   , 0.   , 1.   , 0.72 ],
       [0.   , 0.6  , 0.9  , 0.   , 0.5  , 0.172],
       [1.   , 0.9  , 0.7  , 0.   , 1.   , 0.72 ],
       [1.   , 0.9  , 0.9  , 1.   , 0.5  , 0.489],
       [1.   , 0.15 , 0.8  , 0.   , 0.5  , 0.145],
       [1.   , 0.9  , 0.7  , 0.   , 0.5  , 1.   ],
       [1.   , 0.45 , 0.5  , 0.   , 0.5  , 0.72 ],
       [1.   , 0.9  , 0.8  , 0.   , 1.   , 0.72 ],
       [1.   , 0.9  , 0.7  , 0.   , 1.   , 1.   ],
       [1.   , 0.9  , 0.8  , 0.   , 0.5  , 1.   ],
       [1.   , 0.45 , 0.6  , 0.   , 0.5  , 0.213],
       [1.   , 0.9  , 0.9  , 0.   , 0.5  , 0.72 ],
       [0.   , 0.3  , 0.9  , 0.   , 0.5  , 0.013],
       [0.   , 0.5  , 0.8  , 0.   , 0.5  , 0.213],
       [0.   , 0.9  , 0.8  , 0.   , 0.5  , 0.172],
       [1.   , 0.9  , 1.   , 1.   , 0.5  , 0.311],
       [0.   , 0.9  , 1.   , 0.   , 0.5  , 0.489],
       [0.   , 0.9  , 0.9  , 1.

In [70]:
import torch
import torch.nn as nn
import torch.optim as optim
from matplotlib import pyplot as plt

In [63]:
def loss_fn(y_pred, y):
    loss = 0
    for y1, y2 in zip(y_pred, y):
        loss += pow(abs(y1-y2),2)
    return torch.sqrt(loss/len(y))

In [98]:
if 1:
    history = pd.DataFrame(columns=["Model","nEpoch", "Loss","Loss per epoch" ,"Learning Rate"])

In [96]:
X = torch.tensor(road_train, dtype=torch.float32)
y = torch.tensor(y_train, dtype=torch.float32)


model = nn.Sequential(
    nn.Linear(X.shape[1], 24),
    nn.ReLU(),
    nn.Linear(24,16),
    nn.ReLU(),
    nn.Linear(16,12),
    nn.ReLU(),
    #nn.Linear(12,8),
    #nn.ReLU(),
    nn.Linear(12, 1),
)
print(model)

learining_rate = 0.001
optimizer = optim.Adam(model.parameters(), lr=learining_rate)

n_epochs = 50
batch_size = 10

lpe = np.zeros(n_epochs)

epoch_loss = 0
for j,epoch in enumerate(range(n_epochs)):
    for i in range(0, len(X), batch_size):
        Xbatch = X[i:i+batch_size]
        y_pred = model(Xbatch)
        ybatch = y[i:i+batch_size]
        loss = loss_fn(y_pred, ybatch)
        epoch_loss += loss.item() * Xbatch.size(0)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    epoch_loss /= len(X)  # Średnia strata na epokę
    lpe[j] = epoch_loss
    print(f'Epoka {epoch}, funkcja straty {epoch_loss}')
history.loc[len(history)] = {"Model":model, "nEpoch": n_epochs, "Loss": epoch_loss,"Loss per epoch": lpe, "Learning Rate": learining_rate}

with torch.no_grad():
    y_pred = model(X)
accuracy = (abs(y_pred.round() - y) < 100).float().mean()
print(f"Accuracy {accuracy}")

plt.figure(figsize=(10, 6))
plt.plot(np.arange(n_epochs), lpe, label="Funkcja straty")
plt.xlabel("Epoka")
plt.ylabel("Strata")
plt.title("Wykres funkcji straty w czasie")
plt.legend()
plt.show()

Sequential(
  (0): Linear(in_features=6, out_features=24, bias=True)
  (1): ReLU()
  (2): Linear(in_features=24, out_features=16, bias=True)
  (3): ReLU()
  (4): Linear(in_features=16, out_features=12, bias=True)
  (5): ReLU()
  (6): Linear(in_features=12, out_features=1, bias=True)
)
Epoka 0, funkcja straty 3690.501413652974
Epoka 1, funkcja straty 3418.256773430979
Epoka 2, funkcja straty 2164.64595701077
Epoka 3, funkcja straty 1867.1129573488843
Epoka 4, funkcja straty 1843.934289048495
Epoka 5, funkcja straty 1823.3494098422723
Epoka 6, funkcja straty 1804.195724215956
Epoka 7, funkcja straty 1786.0244935639266
Epoka 8, funkcja straty 1768.6651186478334
Epoka 9, funkcja straty 1752.0848920039982
Epoka 10, funkcja straty 1736.3553203909462
Epoka 11, funkcja straty 1721.6832457698465
Epoka 12, funkcja straty 1708.2036083032322
Epoka 13, funkcja straty 1695.9698085306948
Epoka 14, funkcja straty 1685.0447823913894
Epoka 15, funkcja straty 1675.5001057849088
Epoka 16, funkcja straty 1

KeyboardInterrupt: 

In [82]:
print(type(y_pred), type(y))
accuracy = (abs(y_pred.round() - y) < 500).float().mean()
print(f"Accuracy {accuracy}")

<class 'torch.Tensor'> <class 'torch.Tensor'>
Accuracy 0.39892473816871643


In [97]:
history

,Model,Epoch,Loss,Loss per epoch,Learning Rate
0,"[Linear(in_features=6, out_features=24, bias=T...",NaN,1596.83837,"[3685.3604467248406, 3173.369381994732, 1928.5...",0.001


In [75]:
pd.DataFrame(data={'y_pred': y_pred.round().flatten(), 'y': y.round().flatten()}).head(50)

,y_pred,y
0,1997.0,1200.0
1,3453.0,2850.0
2,5821.0,4700.0
3,1452.0,3500.0
4,2774.0,2600.0
5,5456.0,2834.0
6,1813.0,3749.0
7,2533.0,2250.0
8,919.0,900.0
9,3790.0,3400.0
